In [15]:
from requests import get
import pandas as pd
import pickle, json
import numpy as np

In [16]:
# første sted...TV2!

In [17]:
tv2_rå_svar = get("https://election-api.services.tv2.dk/ep/ep24/candidatetest/answers/999").json()

In [18]:
tv2_spg = [ '1a', '1b', '2a', '2b', '3a', '3b', '4a', '4b', '5a', '5b', '6a', '6b', '7a', '7b', '8a', '8b', '9a', '9b', '10a', '10b', '11a', '11b']

In [19]:
# smæk det hele pænt sammen

tv2_svar = {}
tv2_kandidater = {}
for kandidat in tv2_rå_svar:
    tv2_kandidater[kandidat['id']] = {'navn': kandidat['name'], 'parti': kandidat['partyName'], 'job': kandidat['occupation'], 'alder': kandidat['age']}
    tv2_svar[kandidat['id']] = {x: kandidat['answers'][x]['answer'] for x in tv2_spg if x in kandidat['answers']}
tv2_kandidater = pd.DataFrame(tv2_kandidater).T
tv2_svar = pd.DataFrame(tv2_svar).T

In [20]:
# norm mellem 0 og 1
ma = tv2_svar.max()
mi = tv2_svar.min()
tv2_svar = (tv2_svar-mi)/(ma-mi)

In [21]:
q = tv2_svar.copy()
q['parti'] = [tv2_kandidater.loc[x, 'parti'] for x in tv2_svar.index]
q['navn'] = [tv2_kandidater.loc[x, 'navn'] for x in tv2_svar.index]
q['job'] = [tv2_kandidater.loc[x, 'job'] for x in tv2_svar.index]
q['alder'] = [tv2_kandidater.loc[x, 'alder'] for x in tv2_svar.index]
#q.parti.replace("Konservative", "Det Konservative Folkeparti", inplace=True)

In [3]:
# DR's data er...mere gemt...men jeg fandt en acceptabel måde at hive det ud på via scrapy

objects = []
with (open("TakeTheDR/dr.pickle", "rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break

In [4]:
pd.DataFrame([[len(y) for y in x['kandidat']] for x in objects])[0].value_counts()

0
25    165
0       4
Name: count, dtype: int64

In [5]:
# 165 har svaret.

# Vi beholder kun dem der har svaret.
objects = [x['kandidat'] for x in objects if x['kandidat'][0]]

In [6]:
sum(pd.Series([pd.DataFrame(x[0])['Answer'].sum() for x in objects])==0)
# alle hare svaret

0

In [8]:
#objects = [x for x in objects if pd.DataFrame(x[0])['Answer'].sum() != 0]
#len(objects)
#

165

In [13]:
bogstaver = json.load(open('various.json'))['bogstav_leg']

dr_data = []
for i, dd in enumerate(objects):
    person = {'navn': " ".join([dd[1]['Firstname'], dd[1]['LastName']]), 'parti': bogstaver[dd[1]['CurrentPartyCode']]}
    svar = (pd.DataFrame(dd[0])[['QuestionID', 'Answer']].set_index('QuestionID').T-1)/4
    svar.columns = svar.columns.astype('str')
    svar['navn'] = person['navn']
    svar['parti'] = person['parti']
    svar.index = [f'DR{i}']
    dr_data.append(svar)
dr_df = pd.concat(dr_data)
dr_df = dr_df.replace(-0.25, np.nan)

In [14]:
dr_df

QuestionID,608,609,612,613,614,615,616,617,618,620,...,631,634,636,637,639,641,644,646,navn,parti
DR0,1.0,0.00,1.00,1.00,0.25,0.25,0.00,1.00,0.00,0.75,...,0.25,1.00,1.00,1.00,1.00,0.75,0.25,1.00,Andi Helbo Sejersen,Socialdemokratiet
DR1,1.0,0.25,1.00,1.00,0.75,0.75,0.75,1.00,0.00,0.25,...,0.00,1.00,1.00,1.00,0.75,0.75,0.75,0.75,Magnus Barsøe,Socialdemokratiet
DR2,1.0,0.00,0.75,1.00,1.00,0.75,0.75,1.00,0.00,0.00,...,0.00,0.25,1.00,1.00,0.75,0.25,0.25,1.00,Philip Tarning-Andersen,Radikale Venstre
DR3,1.0,0.25,0.75,1.00,0.75,0.75,0.25,0.75,0.00,0.75,...,0.25,1.00,0.75,1.00,0.25,0.75,0.75,0.25,Christel Schaldemose,Socialdemokratiet
DR4,1.0,0.00,0.75,1.00,1.00,1.00,0.25,1.00,0.00,0.25,...,0.25,0.25,1.00,1.00,0.75,0.75,0.25,1.00,Anne Sophie Callesen,Radikale Venstre
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DR160,1.0,0.00,0.75,1.00,1.00,0.75,0.25,1.00,0.00,0.25,...,0.25,0.75,0.75,1.00,0.75,0.75,0.25,1.00,Jens Frost,Radikale Venstre
DR161,1.0,0.00,0.25,0.75,0.75,0.75,0.75,0.75,0.25,0.25,...,0.25,0.00,0.25,0.75,0.75,0.25,0.25,1.00,Hediye Temiz,Radikale Venstre
DR162,1.0,0.00,0.75,1.00,1.00,0.75,0.25,0.75,0.00,0.25,...,0.25,0.75,1.00,1.00,0.75,0.75,0.25,1.00,Kathrine Olldag,Radikale Venstre
DR163,1.0,0.00,0.75,0.75,0.75,0.75,0.25,1.00,0.00,0.75,...,0.25,0.75,0.75,0.75,0.75,0.25,0.75,0.75,Frederik Aagaard Sørensen,Radikale Venstre


In [22]:
all_data = dr_df.merge(q, on=['navn', 'parti'])

In [23]:
alle_navne = pd.concat([dr_df.navn, q.navn])
folk_mangler = alle_navne[~alle_navne.isin(all_data.navn)]
len(folk_mangler)

37

In [24]:
flatten=lambda l: sum(map(flatten,l),[]) if isinstance(l,list) else [l]
potids = {}
noid = []
for person in folk_mangler:
    navne = person.split()
    potid = []
    for navn in navne:
        liid = folk_mangler[folk_mangler.str.contains(navn)].to_list()
        if len(liid) == 2:
            potid.append(liid)
    potid = list(set(flatten(potid)))
    if potid:
        potid.remove(person)
        if len(potid) == 1:
            potids[person] = potid[0]
        else:
            noid.append(person)
    else:
        noid.append(person)
potids2 = {}
for k, v in potids.items():
    if v not in potids2:
        potids2[k] = v

In [26]:
noid

['Irina Bjørnø',
 'Alexandra Sasha',
 'Dea Kehler',
 'Ivar Nørlund',
 'Stine Steffensen']

In [30]:
potids2

{'Victoria Velasquez': 'Victoria Velásquez',
 'Bob Richard Nielsen': 'Bob Nielsen',
 'Annette Giuranna': 'Anette Giuranna',
 'Benny Bindslev': 'Benny Binderslev',
 'Marianne Lynghøj': 'Marianne Lynghøj Pedersen',
 'Michael Nedersøe': 'Michael Kim Bladt Nedersøe',
 'Anders Vistisen': 'Anders Primdahl Vistisen',
 'Klavs A. Holm': 'Klavs A Holm',
 'Tobias Marney': 'Tobias William Marney',
 'Martin Sibast Laugesen': 'Martin Sibast',
 'Lars Høyer Holmqvist': 'Lars Holmqvist',
 'Emil Njor': 'Emil Jørgensen Njor',
 'Jacob Stryhn': 'Jacob A G Stryhn',
 'Egil Fabritius Hulgaard': 'Egil Hulgaard',
 'Sissel Van Run-Kvist': 'Sissel van Run-Kvist',
 'João A. B. R. Møller': 'João A.B. R. Møller'}

In [33]:
q.navn = q.navn.apply(lambda x: potids2[x] if x in potids2 else x)
dr_df.navn = dr_df.navn.apply(lambda x: potids2[x] if x in potids2 else x)
all_data = dr_df.merge(q, on=['navn', 'parti'])

In [34]:
alle_navne = pd.concat([dr_df.navn, q.navn])
folk_mangler = alle_navne[~alle_navne.isin(all_data.navn)]
len(folk_mangler)

5

In [35]:
folk_mangler

DR16         Irina Bjørnø
DR66      Alexandra Sasha
DR88           Dea Kehler
16600        Ivar Nørlund
16629    Stine Steffensen
Name: navn, dtype: object

In [39]:
# udfyld manglende svar med partiets gennemsnit
dk_spg = ['608', '609', '612', '613', '614', '615', '616', '617', '618', '620',
       '621', '622', '623', '626', '627', '629', '630', '631', '634', '636',
       '637', '639', '641', '644', '646', '1a', '1b', '2a', '2b', '3a', '3b', '4a', '4b', '5a', '5b', '6a', '6b', '7a', '7b', '8a', '8b', '9a', '9b', '10a', '10b', '11a', '11b']
parti_linjen = round(all_data.groupby('parti')[dk_spg].mean()*4)/4  # gennemsnitssvaret per parti, rundet til 1-5 (0-4) svarmulighederne

def fald_ind(x):
    x.loc[x[x.isna()].index] = parti_linjen.loc[x.parti, x[x.isna()].index]
    return x
    
all_data_final = all_data.apply(fald_ind, axis=1)

In [40]:
all_data_final.to_feather("2024_EP_Lasse_data.feather")